In [ ]:
import pandas as pd
!pip install pandas_datareader
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from scipy import stats
from arch import arch_model
import seaborn as sns
from scipy.stats import norm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 63.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
prices = pd.read_csv("/Users/tj_zhang/Library/CloudStorage/OneDrive-SaïdBusinessSchool/1 FE/FE PW4/mfe-practical-work-4-2024-2025.csv.gz", compression="infer")
prices["timestamp"] = pd.to_datetime(prices.timestamp)
prices = prices.set_index("timestamp")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tj_zhang/Library/CloudStorage/OneDrive-SaïdBusinessSchool/1 FE/FE PW4/mfe-practical-work-4-2024-2025.csv.gz'

In [ ]:
print(prices.head(15))
print("...")
print(prices.tail(15))

# Q1

In [ ]:
prices['r'] = np.log(prices['price']).diff()

# Compute squared log-returns
prices['r^2'] = prices['r'] ** 2

# Group by trading day (adjust depending on your trading hours) and sum the squared returns
daily_rv = prices['r^2'].groupby(prices.index.date).sum()

# For example, to compute daily realized volatility:
daily_rsd = np.sqrt(daily_rv)

daily_rv.head()


# Q2

In [ ]:
# Define the sampling intervals in minutes
intervals =  list(range(1, 31))
rv_estimates = []

for i in intervals:
    # Resample the data at the given interval using the last observed price
    resampled_p = prices['price'].resample(f'{i}T').last().dropna()
    
    # Compute log returns
    log_r = np.diff(np.log(resampled_p))
    
    # Compute realized variance as the sum of squared log returns
    rv = np.sum(log_r ** 2)
    rv_estimates.append(rv)

# Plot the signature plot
plt.plot(intervals, rv_estimates, marker='o')
plt.xlabel('Sampling Interval (mins)')
plt.ylabel('Realized Variance')
plt.title('Signature Plot for Realized Variance')
plt.show()

# Q3

In [ ]:
# realized_variance should be list of RV with the optimal frequency we picked

# below are chatgpt's answer

# start with taking ln RV, then plot residuals to see if residual is normal, if not normal, model RV instead

interval = 14  # Optimal interval is 14 minutes
prices_resampled = prices['price'].resample(f'{interval}T').last().dropna()

# Convert to DataFrame for grouping
prices_resampled = prices_resampled.to_frame()

# Extract the date part for grouping
prices_resampled['date'] = prices_resampled.index.date

# Compute the log return within each date group
log_returns = np.log(prices_resampled.groupby('date')['price']).diff()

# Drop NaN values that result from the first difference in each group
log_returns.dropna(inplace=True)

# Compute squared log returns (Realized Variance)
rv = log_returns ** 2

# Sum squared returns within each day to get daily RV
daily_rv = rv.groupby(prices_resampled.index.date).sum()

rv_df = pd.DataFrame(daily_rv, columns=['RV'])

rv_df['RV_1'] = rv_df['RV'].shift(1)  # 1-day lag
rv_df['RV_5'] = rv_df['RV'].rolling(window=5).mean().shift(1)  # 5-day rolling mean, shifted
rv_df['RV_22'] = rv_df['RV'].rolling(window=22).mean().shift(1)  # 22-day rolling mean, shifted

# ln variables
rv_df['ln_RV'] = np.log(rv_df['RV'])
rv_df['ln_RV_1'] = np.log(rv_df['RV_1'])
rv_df['ln_RV_5'] = np.log(rv_df['RV_5'])
rv_df['ln_RV_22'] = np.log(rv_df['RV_22'])

# Drop missing values due to lagging
rv_df = rv_df.dropna()

# Define dependent (Y) and independent (X) variables
y = rv_df['ln_RV']
X = rv_df[['ln_RV_1', 'ln_RV_5', 'ln_RV_22']]

# 2. Split data into training (50%) and testing (50%) sets
split_index = int(len(daily_rv) * 0.5)
train_data = daily_rv[:split_index]  # 50% for training
test_data = daily_rv[split_index:]  # 50% for testing

# 3. Construct HAR model
X = sm.add_constant(X)

# Fit the OLS regression model
har_model = sm.OLS(y, X).fit()

# Print results
print(har_model.summary())

# 4. Make predictions and evaluate performance
pred_1 = har_model.predict(sm.add_constant(pd.concat([train_data.shift(i) for i in range(1, 2)], axis=1)))

# For evaluation, you can calculate RMSE or any other relevant metric
from sklearn.metrics import mean_squared_error
rmse_1 = np.sqrt(mean_squared_error(test_data[:len(pred_1)], pred_1))
rmse_5 = np.sqrt(mean_squared_error(test_data[:len(pred_5)], pred_5))
rmse_22 = np.sqrt(mean_squared_error(test_data[:len(pred_22)], pred_22))

print(f'RMSE for 1-lag HAR model: {rmse_1}')
print(f'RMSE for 5-lag HAR model: {rmse_5}')
print(f'RMSE for 22-lag HAR model: {rmse_22}')

In [ ]:
# plot residual to see whether to log

residuals = har_model.resid

residual_mean = residuals.mean()
residual_std = residuals.std()

# Generate values for the normal distribution
x = np.linspace(residuals.min(), residuals.max(), 100)
y = norm.pdf(x, residual_mean, residual_std)

# Plot residual histogram
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=False, bins=30, stat="density", color='blue', alpha=0.6, label="Residuals")

# Overlay normal distribution
plt.plot(x, y, 'r-', lw=2, label="Normal Dist (same mean & variance)")

# Labels and legend
plt.title("Residuals Distribution with Normal Overlay")
plt.xlabel("Residuals")
plt.ylabel("Density")
plt.legend()
plt.show()

# if residual doesn't look like normal we should not use ln

# Q4

In [ ]:
# If asymmetry is significant, use GJR-GARCH(1,1,1).

# Use daily close-to-close log returns
daily_returns = np.log(prices["price"].resample("1D").last()).diff().dropna()

# Split dataset into 50% training
train_size = int(len(daily_returns) * 0.5)
train_returns = daily_returns.iloc[:train_size]

# Fit GARCH(1,1) model
garch_model = arch_model(train_returns, vol="Garch", p=1, q=1, mean="Constant")
garch_fit = garch_model.fit()
print(garch_fit.summary())

# Q5

In [ ]:
# check if pseudo return is right
pseudo_returns = daily_returns / np.sqrt(realized_variance)

# Fit GARCH model to pseudo-returns
train_pseudo_returns = pseudo_returns.iloc[:train_size]

garch_pseudo_model = arch_model(train_pseudo_returns, vol="Garch", p=1, q=1, mean="Constant")
garch_pseudo_fit = garch_pseudo_model.fit()
print(garch_pseudo_fit.summary())

# Q6